# Obtain AOU and calculate AOU_bar from FOSI hindcast

In [1]:
import xarray as xr
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
a1,a2,b1,b2 = [240,380,125,320]

### DPLE hindcast

In [3]:
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
filename = f'{datadir}/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.AOU.024901-031812.nc'
ds_hind = xr.open_dataset(filename)
ds_hind

<xarray.Dataset>
Dimensions:               (z_t: 60, z_w: 60, transport_reg: 2, nlat: 384, nlon: 320, z_w_top: 60, z_w_bot: 60, transport_comp: 5, lat_aux_grid: 395, z_t_150m: 15, moc_comp: 3, moc_z: 61, time: 840, d2: 2)
Coordinates:
  * z_t                   (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_w                   (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
    TLAT                  (nlat, nlon) float64 ...
  * z_w_top               (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot               (z_w_bot) float32 1e+03 2e+03 ... 5.25e+05 5.5e+05
    TLONG                 (nlat, nlon) float64 ...
  * lat_aux_grid          (lat_aux_grid) float32 -79.49 -78.95 ... 89.47 90.0
  * z_t_150m              (z_t_150m) float32 500.0 1.5e+03 ... 1.35e+04 1.45e+04
    ULAT                  (nlat, nlon) float64 ...
  * moc_z                 (moc_z) float32 0.0 1e+03 2e+03 ... 5.25e+05 5.5e+05
    ULONG                 (nlat, nlon) float64 ...
  * time                  (time) object 0249-02-01 00:00:00 ... 0319-01-01 00...
Dimensions without coordinates: transport_reg, nlat, nlon, transport_comp, moc_comp, d2
Data variables: (12/54)
    transport_regions     (transport_reg) |S256 ...
    ANGLET                (nlat, nlon) float64 ...
    HUW                   (nlat, nlon) float64 ...
    HUS                   (nlat, nlon) float64 ...
    transport_components  (transport_comp) |S256 ...
    latent_heat_vapor     float64 ...
    ...                    ...
    omega                 float64 ...
    DXU                   (nlat, nlon) float64 ...
    DXT                   (nlat, nlon) float64 ...
    ppt_to_salt           float64 ...
    time_bound            (time, d2) object ...
    AOU                   (time, z_t, nlat, nlon) float32 ...
Attributes:
    nsteps_total:    2263750
    tavg_sum:        2678400.0
    title:           g.DPLE.GECOIAF.T62_g16.009.chey
    start_time:      This dataset was created on 2018-06-06 at 16:42:05.9
    history:         none
    Conventions:     CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-curr...
    source:          CCSM POP2, the CCSM Ocean Component
    cell_methods:    cell_methods = time: mean ==> the variable values are av...
    calendar:        All years have exactly  365 days.
    tavg_sum_qflux:  2678400.0
    contents:        Diagnostic and Prognostic Variables
    revision:        $Id: tavg.F90 41939 2012-11-14 16:37:23Z mlevy@ucar.edu $

## calculate AOU mean

In [4]:
nmon = ds_hind.dims['time']
aa = xr.DataArray(data=np.ones(nmon),coords=dict(time=('time',ds_hind.coords['time'].values)),dims='time')

In [5]:
dso = xr.Dataset()
dso['AOU'] = ds_hind['AOU'].isel(z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))
AOU_mean = dso['AOU'].mean(dim='time')
dso['AOU_mean'] = aa*AOU_mean

In [6]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dso.load()

<xarray.Dataset>
Dimensions:   (z_t: 35, nlat: 140, nlon: 195, time: 840)
Coordinates:
  * z_t       (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT      (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    TLONG     (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
    ULAT      (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    ULONG     (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
  * time      (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables:
    AOU       (time, z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan
    AOU_mean  (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan

In [7]:
%%time
dso.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_AOU_and_AOU_mean_1948-2017_monthly.nc', mode='w')

CPU times: user 45.8 s, sys: 4.38 s, total: 50.2 s
Wall time: 53.1 s
